In [111]:
#Import dependencies
import pandas as pd
import pymongo
import requests
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager


In [112]:
# configure ChromeDriver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/henrytran/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


# NASA MARS NEWS 

In [113]:
def mars_news(): 
   
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)

    html = browser.html
    new_soup = soup(html, 'html.parser')

    #article_container = new_soup.select('ul.item_list')
    article_container = new_soup.find('ul', class_ = 'item_list')


    headline_date = article_container.find('div', class_ = 'list_date').text
    news_title = article_container.find('div', class_ = 'content_title').find('a').text
    news_p = article_container.find('div', class_ = 'article_teaser_body').text

    return news_title, news_p

# JPL MARS SPACE IMAGES

##### There have been changes to the nasa site. As such, I set the code below to the first pic

In [114]:
 def featured_image():   
    url = 'https://www.jpl.nasa.gov/images/ingenuity-helicopter-on-mars-illustration'
    browser.visit(url)

    html = browser.html
    img_soup = soup(html, 'html.parser')

    #Method 1
#     try:
#         img_element = img_soup.find('article', class_ = 'carousel_item')['style']

#         img_rel_url = img_element.replace("backround-image: url('",'')
#         img_rel_url = img_rel_url.replace("');",'')

#         print(img_rel_url)
#     except Exception as e:
#         print(e)
        #Method 2
    try:
#             full_image_elem = browser.find_by_id('full_image')[0]
#             full_image_elem.click()

#             #html = browser.html
#             #img_soup = soup(html, 'html.parser')
        img_rel_url = img_soup.find(class_="BaseImage object-scale-down")['src']
        print(img_rel_url)
        featured_image_url = f'{img_rel_url}'
#             print(featured_img_url)
    except Exception as e:
#         print(e)
        return featured_image_url

In [115]:

# url = 'https://www.jpl.nasa.gov/images/ingenuity-helicopter-on-mars-illustration'
# browser.visit(url)

# html = browser.html
# img_soup = soup(html, 'html.parser')

# #Method 1
# #     try:
# #         img_element = img_soup.find('article', class_ = 'carousel_item')['style']

# #         img_rel_url = img_element.replace("backround-image: url('",'')
# #         img_rel_url = img_rel_url.replace("');",'')

# #         print(img_rel_url)
# #     except Exception as e:
# #         print(e)
#     #Method 2
# try:
# #     full_image_elem = browser.find(class_='BaseImage  object-contain')[0]
# #     full_image_elem.click()

#     #html = browser.html
#     #img_soup = soup(html, 'html.parser')
#     img_rel_url = img_soup.find('id="96834")['scr']
#     print(img_rel_url)

#     featured_image_url = f'{img_rel_url}'
#     print(featured_img_url)
# except Exception as e:
#     print(e)


In [116]:
img_rel_url = img_soup.find(class_="BaseImage object-scale-down")['src']
img_rel_url
# src =img_rel_url.find('src')

'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24466.width-1024.jpg'

In [117]:
featured_image()

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24466.width-1024.jpg



# Mars Facts

In [118]:
def mars_facts():
    
    url = 'https://space-facts.com/mars/'
    browser.visit(url)

    mars_facts_df = pd.read_html(url)
    mars_facts_df = mars_facts_df[0]
    mars_facts_df.columns = ['Description', 'Mars']
    mars_facts_df

    mars_facts_html = mars_facts_df.to_html(classes='table table-striped')
    
    return mars_facts_html

In [119]:
html = browser.html
facts_soup =soup(html, 'html.parser')
facts_soup.find(id='tablepress-p-mars')

# Mars 

# Insert into Mongo DB

In [120]:
def scrape_all():

    news_title, news_p = mars_news()
    featured_img_url = featured_image()
    mars_facts_html = mars_facts()

    #Assemble the document to insert into the database
    nasa_document = {
        'news_title': news_title,
        'news_paragraph': news_p,
        'featured_img_url': featured_img_url,
        'mars_fact_html': mars_facts_html
    }

    return nasa_document

In [121]:
scrape_all()


https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24466.width-1024.jpg


{'news_title': "New Study Challenges Long-Held Theory of Fate of Mars' Water",
 'news_paragraph': 'The new science results indicate that a large quantity of the Red Planet’s water is trapped in its crust rather than having escaped into space.',
 'featured_img_url': None,
 'mars_fact_html': '<table border="1" class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</

In [122]:
#Connect to MongoDB

conn = 'mongodb://localhost:27017'
client =pymongo.MongoClient(conn)

db = client.mars_app

mars = db.mars

data_document = scrape_all()


mars.update_one({},{'$set': data_document}, upsert=True)


https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24466.width-1024.jpg
